In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import random
import pickle

# Load data from intents.json
with open('intents.json') as file:
    data = json.load(file)

# Extract data for training
training_sentences = []
training_labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

# Initialize label encoder and fit it to the training labels
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

# Set hyperparameters
vocab_size = 1000
embedding_dim = 32
max_len = 20
oov_token = "<OOV>"
num_classes = len(set(training_labels))
epochs = 500

# Initialize and fit tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)

# Create word index from tokenizer
word_index = tokenizer.word_index

# Convert training sentences to sequences and pad them
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

# Define and compile the model with LSTM
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

# Save the trained model
model.save("chat_model_best_accuracy")

# Save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 32)            32000     
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 8)                 264       
                                                                 
Total params: 59,176
Trainable params: 59,176
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
2/2 [==============================] - 4s 18ms/step - loss: 2.0803 - accuracy: 0.0606
Epoch 2/500
2/2 [==============================] - 0s 0s/step - loss: 2.0748 - accurac

INFO:tensorflow:Assets written to: chat_model_best_accuracy\assets


INFO:tensorflow:Assets written to: chat_model_best_accuracy\assets


Start messaging with the bot (type 'quit' to stop)!


In [ ]:

import json 
import numpy as np
from sklearn.preprocessing import LabelEncoder
import colorama 
from colorama import Fore, Style, Back
import random
import pickle
from tensorflow import keras

colorama.init()
# load intents.json file
with open("intents.json") as file:
    data = json.load(file)

# Define a function to start the chat
def chat():
    # load trained model
    try:
        model = keras.models.load_model('chat_model_best_accuracy')
    except:
        # If loading fails due to optimizer, try loading with custom objects
        model = keras.models.load_model('chat_model_best_accuracy', compile=False)
        model.compile(optimizer='adam')  # Recompile with a standard optimizer
    
    # rest of your code...

    # Load the tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # Load the label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # Parameters
    max_len = 20

    # Loop to take user input and get bot response
    while True:
        # Get user input
        inp = input("You: ")

        # Check if the user wants to quit
        if inp.lower() == "quit":
            break

        # Preprocess the input and predict the intent
        result = model.predict(
            keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                                       truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])[0]

        # Get a random response from the list of responses for the predicted intent
        for intent in data['intents']:
            if intent['tag'] == tag:
                response = random.choice(intent['responses'])
                print("Bot:", response)

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()


Start messaging with the bot (type quit to stop)!
You: Hello
1/1 [==============================] - 1s 1s/step
Bot: Hi
You: who are you
1/1 [==============================] - 0s 74ms/step
Bot: I'm Uzair, an Artificial Intelligent bot
You: please help me
1/1 [==============================] - 0s 38ms/step
Bot: Tell me your problem to assist you
You: i want to create account
1/1 [==============================] - 0s 40ms/step
Bot: You can just easily create a new account from our web site
You: Thank you
1/1 [==============================] - 0s 58ms/step
Bot: My pleasure


In [ ]:
Thank you